In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tp2datostrain/test.csv
/kaggle/input/pingsound/PING - Sound effect.mp3


In [58]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from IPython.display import Audio
sound_file = '/kaggle/input/pingsound/PING - Sound effect.mp3'

In [59]:
test = pd.read_csv("../input/tp2datostrain/test.csv")
display(test.shape)
train = pd.read_csv("../input/tp2datostrain/train.csv")
display(train.shape)

(60000, 22)

(240000, 23)

In [60]:
print(len(test["ciudad"].unique()))
print(len(train["ciudad"].unique()))
ciudades = list(train["ciudad"].unique())
tipos_de_propiedad = list(train["tipodepropiedad"].unique())
provincia = list(train["provincia"].unique())
categorias  = [provincia, tipos_de_propiedad, ciudades]

577
876


In [105]:
#Elijo si voy a hacer train o test!
TRAIN = True
CV = True and TRAIN

# Arreglo el DF

In [109]:
df = train if TRAIN else test
df["fecha"] = pd.to_datetime(df["fecha"])

In [111]:
#RELLENO LOS IDS CON LO QUE MAS APARECE EN LA CIUDAD
def obtenerMasFrecuente(x):
    x = pd.Series.mode(x)
    s = x.loc[x>0]
    if len(s.index)<=0:
        return np.nan
    return s.value_counts().index[0]
print("Nulos idzona: ", df["idzona"].isna().value_counts())
print("Nulos coidad: ", df["ciudad"].isna().value_counts())

aux = df[["ciudad", "idzona"]].copy()
aux["ciudad"].fillna("dummy", inplace=True)
#aux["idzona"].fillna(-1, inplace=True)
idzonaMasFrecuentePorCiudad = aux.groupby('ciudad')["idzona"].transform(lambda x: obtenerMasFrecuente(x))
idzonaMasFrecuentePorCiudad

Nulos idzona:  False    239521
Name: idzona, dtype: int64
Nulos coidad:  False    239521
Name: ciudad, dtype: int64


0         50003995.0
1            24481.0
2            48638.0
3            53666.0
4            47735.0
             ...    
239995       53666.0
239996       51929.0
239997    50003995.0
239998       24448.0
239999       23734.0
Name: idzona, Length: 239521, dtype: float64

In [112]:
df.idzona.fillna(idzonaMasFrecuentePorCiudad, inplace = True)
print("Nulos idzona: ", df["idzona"].isna().value_counts())
#DROPEO LAS COLUMNAS QUE QUEDARON SIN NADA si estoy entrenando o relleno con la mas comun si estoy testeando
if TRAIN:
    df.dropna(subset=['idzona'], inplace = True)
else:
    df.fillna(df['idzona'].value_counts().idxmax(), inplace = True)
print("Nulos idzona: ", df["idzona"].isnull().value_counts())
df.shape

Nulos idzona:  False    239521
Name: idzona, dtype: int64
Nulos idzona:  False    239521
Name: idzona, dtype: int64


(239521, 24)

In [113]:
def rellenar_con_X_por_grupo(df, grupo, rellenar, func):
    valoresAgrupados = df.groupby(grupo)[rellenar].transform(func)
    df[rellenar].fillna(valoresAgrupados, inplace=True)

In [114]:
#Reemplazo las habitaciones por la mediana de la zona
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
rellenar_con_X_por_grupo(df, "idzona", "habitaciones", lambda x: x.median())
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
#Relleno el resto con las mas frecuentes
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])
#Para la antiguedad relleno con el promedio de la zona
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())
medianaAntiguedadXZona = df.groupby("idzona")["antiguedad"].transform(lambda x: x.median())
df["antiguedad"].fillna(medianaAntiguedadXZona, inplace = True)
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())
#Reemplazo los banos por 1 que es lo minimo posible
df.banos.fillna(1, inplace=True)
#Relleno garages con 0 porque es lo mas probable
df.garages.fillna(0, inplace = True)
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
#COMO QUEDAN NULOS LOS RELLENO CON la mediana (porque la mediana es un valor, y es menos sensible a valores extremos) por ZONA / CIUDAD
promedioMetrosCubiertosXZona = df.groupby("idzona")["metroscubiertos"].transform(lambda x: x.median())
df["metroscubiertos"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
promedioMetrosTotalesXZona = df.groupby("idzona")["metrostotales"].transform(lambda x: x.median())
df["metrostotales"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())

Nulos habitaciones:  False    239521
Name: habitaciones, dtype: int64
Nulos habitaciones:  False    239521
Name: habitaciones, dtype: int64
Nulos antiguedad:  False    237924
True       1597
Name: antiguedad, dtype: int64
Nulos antiguedad:  False    237924
True       1597
Name: antiguedad, dtype: int64
Nulos metrostotales:  False    239521
Name: metrostotales, dtype: int64
Nulos metroscubiertos:  False    238777
True        744
Name: metroscubiertos, dtype: int64
Nulos metroscubiertos:  False    238777
True        744
Name: metroscubiertos, dtype: int64
Nulos metrostotales:  False    239521
Name: metrostotales, dtype: int64


In [115]:
df["swapear"] = df["metrostotales"] < df["metroscubiertos"]
df_metros_arreglados = df.copy()
aux = df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metrostotales"]
if (not aux.empty):
    df_metros_arreglados.loc[df_metros_arreglados["swapear"],"metrostotales"] = \
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"]
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"] = aux
df_metros_arreglados.drop(columns="swapear", inplace=True)
(df_metros_arreglados["metrostotales"] < df_metros_arreglados["metroscubiertos"]).value_counts()
df = df_metros_arreglados

In [116]:
df.garages.fillna(0, inplace = True)

In [117]:
df["nro_publicaciones_dia"] = df.groupby(pd.Grouper(key='fecha', freq='D'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_semana"] = df.groupby(pd.Grouper(key='fecha', freq='W'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_5_dias_habiles"] = df.groupby(pd.Grouper(key='fecha', freq='B'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_mes"] = df.groupby(pd.Grouper(key='fecha', freq='M'))["id"].transform(lambda x: len(x.index))

### Agrego features del texto

In [118]:
df["titulo_descripcion"] = df["titulo"].astype(str) + " " + df["descripcion"].astype(str)
#Como tiene algunos nulos los relleno con cadenas vaicas
df["titulo_descripcion"].fillna("",inplace = True)
#REMUEVO LA PUNTUACION
import string
print(string.punctuation)
tabla = {}
for c in string.punctuation + "¿¡":
    tabla[ord(c)] = None
def sacar_puntuacion(s):
    return s.translate(tabla)
df["titulo_descripcion"] = df["titulo_descripcion"].apply(sacar_puntuacion)
#Agrego los primeros features
df["nro_palabras"] = df["titulo_descripcion"].apply(lambda x: len(x.split(" ")))
df["nro_caracteres"] = df["titulo_descripcion"].apply(lambda x: len(x))
df["long_prom_palabra"] = df["nro_caracteres"] / df["nro_palabras"]

def llenarDiccionario(diccionario,descripcion):
    for palabra in descripcion.split():
        diccionario[palabra] = diccionario.get(palabra,0) + 1
        
def generarDiccionario(df2):
    dicc = {}
    for descripcion in df2['titulo_descripcion']:
        if isinstance(descripcion,str): llenarDiccionario(dicc,descripcion)
    return dicc
contador_palabras = generarDiccionario(df)

#Elimino las stopwords = palabras vacias que no suman nada
from nltk.corpus import stopwords
palabras_vacias = stopwords.words('spanish')

for palabra  in palabras_vacias:
    if palabra in contador_palabras:
        del contador_palabras[palabra]
ordenados = sorted(contador_palabras.items(),key = lambda x: -x[1])
top_50 = dict(ordenados[:50])

#Nuevos features
def contar_palabras_en_top(texto):
    palabras = texto.split()
    cont = 0
    for palabra in palabras:
        if palabra in top_50:
            cont += 1
    return cont
df["nro_palabras_mas_comunes"] = df["titulo_descripcion"].apply(contar_palabras_en_top)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [119]:
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,precio,nro_publicaciones_dia,nro_publicaciones_semana,nro_publicaciones_5_dias_habiles,nro_publicaciones_mes,titulo_descripcion,nro_palabras,nro_caracteres,long_prom_palabra,nro_palabras_mas_comunes
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,5.0,2.0,1.0,...,2273000.0,161,1056,451,4526,depto tipo a402 depto interior de 8015m2 const...,29,178,6.137931,15
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,3600000.0,100,487,171,2015,condominio horizontal en venta pentre sonora y...,98,691,7.051020,26
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,1200000.0,384,1467,715,5550,casa en venta urbi 3 recamaras tonala descripc...,218,1339,6.142202,24
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,650000.0,43,308,116,1103,casa sola en toluca zinacantepec con credito i...,64,418,6.531250,6
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,1150000.0,543,2237,543,6761,paseos del sol bonito departamento en excelent...,19,113,5.947368,1


# Preprocesador

In [120]:
features = ['idzona',
            "metrostotales", "metroscubiertos", 
            'garages',
            'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
            'nro_publicaciones_dia',
            'nro_publicaciones_semana',
            'nro_publicaciones_5_dias_habiles',
            'nro_publicaciones_mes',
            'antiguedad',
            'habitaciones',
            "nro_palabras", "nro_caracteres", "long_prom_palabra", "nro_palabras_mas_comunes",
            'provincia','tipodepropiedad', 'ciudad',
            ]

In [121]:
transformador_categorias = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                         OneHotEncoder(handle_unknown='ignore', categories = categorias))
#transformador_categorias_ciudades = make_pipeline(SimpleImputer(strategy='most_frequent'),
#                                         OneHotEncoder(handle_unknown='ignore', categories = [ciudades]))
#transformador_categorias_propiedad = make_pipeline(SimpleImputer(strategy='most_frequent'),
#                                         OneHotEncoder(handle_unknown='ignore', categories = [tipos_de_propiedad]))
transformador_numeros = make_pipeline(SimpleImputer())

In [122]:
def obtenerFeaturesCategoricos(features):
    l = []
    for feature in features:
        if not np.issubdtype(df[feature].dtype, np.number):
            l.append(feature)
    return l

num_features = list(df[features].select_dtypes(include=[np.number]).columns)

cat_features = list(obtenerFeaturesCategoricos(features))
print ("Features numericos:", num_features, '\nFeatures categoricos: ', cat_features)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

Features numericos: ['idzona', 'metrostotales', 'metroscubiertos', 'garages', 'gimnasio', 'centroscomercialescercanos', 'usosmultiples', 'piscina', 'escuelascercanas', 'nro_publicaciones_dia', 'nro_publicaciones_semana', 'nro_publicaciones_5_dias_habiles', 'nro_publicaciones_mes', 'antiguedad', 'habitaciones', 'nro_palabras', 'nro_caracteres', 'long_prom_palabra', 'nro_palabras_mas_comunes'] 
Features categoricos:  ['provincia', 'tipodepropiedad', 'ciudad']


In [145]:
model = xgb.XGBRegressor(early_stopping_rounds=250, #n_estimators=1000
                        learning_rate=0.01, 
                         max_depth=6,
                         subsample = 0.8,
                         gamma = 2,
                         colsample_bytree = 0.8, colsample_bylevel = 0.8, colsample_bynode = 0.8,
                     objective='reg:squarederror',                     
                     tree_method='gpu_hist', gpu_id=0) if TRAIN else model
#model = xgb.XGBRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, subsample_by_tree=0.5, colsample_by_col=0.5, min_child_weight=3,tree_method='gpu_hist', gpu_id=0)

SyntaxError: invalid syntax (<ipython-input-145-892a1835ac55>, line 7)

# Lo entreno

In [146]:
y = df.precio.copy() if TRAIN else None

X = df[features].copy()
if CV:
    Xaux, Txaux, yaux, Tyaux = train_test_split(X, y,train_size=0.8, test_size=0.2, random_state = 0)
X = Xaux if CV else X
y = yaux if CV else y
Tx = Txaux if CV else None
Ty = Tyaux if CV else None

In [147]:
if (TRAIN):
    nuevo_pipeline = make_pipeline(preprocessor, model, verbose = True)
    nuevo_pipeline.fit(X, y)
else:
    nuevo_pipeline = pipeline
pipeline = nuevo_pipeline
#model.get_booster().feature_names = features

[Pipeline] . (step 1 of 2) Processing columntransformer, total=  36.0s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total=   3.8s


In [148]:
len(X.columns), len(Tx.columns)

(22, 22)

In [149]:
Tx = Tx if (TRAIN and CV) else X #Si estoy en TRAIN y con Cross Validatrion uso el Tx que ya armé. Sino el X entero
print(Tx.shape)
predicciones = pipeline.predict(Tx)

(47905, 22)


In [150]:
if (TRAIN):
    Ty = Ty if CV else y
    print("MAE XGBoost: ", mean_absolute_error(Ty, predicciones))

MAE XGBoost:  1072088.968375561


In [151]:
import matplotlib.pyplot as plt
#fig, ax = plt.subplots(1,1,figsize=(10,10))
#xgb.plot_importance(model.get_booster(), ax = ax)
#for col,score in zip(X.columns,model.feature_importances_):    print(col,score)

In [152]:
Audio(sound_file, autoplay=True)

In [153]:
# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "resultados.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


if not TRAIN:
    res = pd.DataFrame(predicciones, index=test.id, columns=['precio'])
    res.reset_index(inplace=True)
    res.columns = ["id", "target"]
    display(res.head())
    # create a link to download the dataframe
    display(create_download_link(res))

In [ ]:
from sklearn.metrics import accuracy_score
Tx2 = preprocessor.fit_transform(Tx)
X2 = preprocessor.fit_transform(X)

pipeline.fit(X,y, xgbregressor__eval_set=[(X2, y.to_numpy()), (Tx2, Ty.to_numpy())])
preds = pipeline.predict(Tx)
predictions = [round(value) for value in preds]
# evaluate predictions
#accuracy = accuracy_score(Ty, preds)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("MAE XGBoost: ", mean_absolute_error(Ty, predicciones))

In [ ]:
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
ax.legend()
plt.ylabel('rmse')
plt.title('XGBoost rmse')
# plot classification error
"""
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()
"""

In [ ]:
"""
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(xgbregressor__learning_rate=learning_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(pipeline, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score'] *-1
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot
plt.errorbar(learning_rate, means, yerr=stds)
plt.title("XGBoost learning_rate vs Squared error")
plt.xlabel('learning_rate')
plt.ylabel('Squared error')
"""

In [ ]:
Audio(sound_file, autoplay=True)